In [1]:
%load_ext twisted_ipython
%autoawait twisted

In [2]:
import attr
from twisted.internet import reactor
from txdbus import client
from txdbus.interface import DBusInterface, Method, Property, Signal
from txdbus.objects import DBusObject, DBusProperty

from pyxsession.twisted.util import returns_deferred
from pyxsession.dbus.marshmallow.schema import DBUS_SCHEMA
from pyxsession.dbus.marshmallow.fields import (
    DBusField, Int32, UInt32, Str, List, Tuple
)
from pyxsession.dbus.service import Service

      
@attr.s
class Payload:
    message = attr.ib(default='', metadata={DBUS_SCHEMA: Str()})



In [3]:
echo_service = Service('Echo', 'org.jfhbrook', 'EchoIface')

@echo_service.method([Payload], Str())
def echo(payload):
    return f'You said: {payload.message}'

In [4]:
conn = await client.connect(reactor)

In [5]:
server = await echo_service.server(conn)

server

1

In [6]:
client = await echo_service.client(conn)

client

In [7]:
client.call('echo', Payload('hello'))

KeyError: 'message'